In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/ML_LSTM/dataset/ /content/

In [ ]:
# coding: utf-8
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.recurrent import LSTM
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras import metrics

In [ ]:
# CSVファイルからデータをDataFrame型で取得する
def get_df_data():
    df_data = pd.read_csv('dataset/dataset_train.csv', index_col=0, parse_dates=True, skiprows = 1, encoding = 'utf8', header=None)
    df_data.columns = [
        'sensor1',
        'sensor2',
        'sensor3',
        'sensor4',
        'feed1',
        'feed2',
        'feed3',
        'feed4',
    ]

    return df_data

In [ ]:

# モデルに読み込ませるデータを生成する
def generate_data(data, length_per_unit, dimension):
    # DataFrame→array変換
    data_array = data.values
    # 時系列データを入れる箱
    sequences = []
    # 正解データを入れる箱
    target = []
    # 正解データの日付を入れる箱
    target_date = []

    # 一グループごとに時系列データと正解データをセットしていく
    for i in range(0, data_array.shape[0] - length_per_unit):
        sequences.append(data_array[i:i + length_per_unit])
        target.append(data_array[i + length_per_unit])
        target_date.append(data[i + length_per_unit: i + length_per_unit + 1].index.strftime('%Y/%m/%d'))

    # 時系列データを成形
    X = np.array(sequences).reshape(len(sequences), length_per_unit, dimension)
    # 正解データを成形
    Y = np.array(target).reshape(len(sequences), dimension)
    # 正解データの日付データを成形
    Y_date = np.array(target_date).reshape(len(sequences), 1)

    return (X, Y, Y_date)


In [ ]:

# モデルリストを取得する
def get_model_list(input_shape):
    # 実行するモデル一覧
    model_list = [
        ['LSTM_1', Sequential([
            LSTM(100, input_shape=input_shape), 
            Dense(4), 
            Activation("linear")])], 
        ['LSTM_2', Sequential([
            LSTM(300, input_shape=input_shape), 
            Dense(4), 
            Activation("linear")])], 
    ]

    return model_list


In [ ]:
# 扱う特徴量
FEATURE_VALUE = ['feed1','feed2','feed3','feed4']
# 次元数
DIMENSION = len(FEATURE_VALUE)

In [ ]:
# データを取得
df_data = get_df_data()
print(df_data)
# 学習用データを取得(日付順にソート)
df_data_train = df_data.loc['1993-01-01':'2020-12-10', FEATURE_VALUE]
df_data_train = df_data_train.sort_index()
df_data_train = df_data_train.dropna() # 欠損値のある行を取り除く
print(df_data_train)

            sensor1  sensor2  sensor3  sensor4  feed1  feed2  feed3  feed4
0                                                                         
1993-01-01        0        0        0        0     10      0      0      0
1993-01-02        0        0        0        0     10      0      0      0
1993-01-03        0        0        0        0     10      0      0      0
1993-01-04       10        0        0        0      8      2      0      0
1993-01-05        8        2        0        0      7      3      0      0
...             ...      ...      ...      ...    ...    ...    ...    ...
2020-12-06        1        1        7        1      0      0      2      8
2020-12-07        1        1        5        3      0      0      4      6
2020-12-08        1        1        1        7      0      0      0     10
2020-12-09        1        1        1        7      0      0      0     10
2020-12-10        1        1        1        7      0      0      0     10

[10206 rows x 8 columns]

In [ ]:
# 一つの時系列データの長さ
LENGTH_PER_UNIT = 10
X_train, Y_train, Y_train_date = generate_data(df_data_train, LENGTH_PER_UNIT, DIMENSION)
# 正規化
X_train = X_train / np.nanmax(np.abs(X_train))
print(X_train)

# 入力の形状
input_shape=(LENGTH_PER_UNIT, DIMENSION)

model_list = get_model_list(input_shape)

# 最適化手法の設定
opt = optimizers.Adam()

[[[1.  0.  0.  0. ]
  [1.  0.  0.  0. ]
  [1.  0.  0.  0. ]
  ...
  [0.2 0.3 0.5 0. ]
  [0.1 0.4 0.5 0. ]
  [0.1 0.3 0.6 0. ]]

 [[1.  0.  0.  0. ]
  [1.  0.  0.  0. ]
  [0.8 0.2 0.  0. ]
  ...
  [0.1 0.4 0.5 0. ]
  [0.1 0.3 0.6 0. ]
  [0.1 0.2 0.7 0. ]]

 [[1.  0.  0.  0. ]
  [0.8 0.2 0.  0. ]
  [0.7 0.3 0.  0. ]
  ...
  [0.1 0.3 0.6 0. ]
  [0.1 0.2 0.7 0. ]
  [0.1 0.1 0.8 0. ]]

 ...

 [[0.3 0.5 0.2 0. ]
  [0.2 0.4 0.4 0. ]
  [0.  0.3 0.4 0.1]
  ...
  [0.  0.  0.4 0.6]
  [0.  0.  0.2 0.8]
  [0.  0.  0.4 0.6]]

 [[0.2 0.4 0.4 0. ]
  [0.  0.3 0.4 0.1]
  [0.  0.4 0.5 0.1]
  ...
  [0.  0.  0.2 0.8]
  [0.  0.  0.4 0.6]
  [0.  0.  0.  1. ]]

 [[0.  0.3 0.4 0.1]
  [0.  0.4 0.5 0.1]
  [0.  0.4 0.5 0.1]
  ...
  [0.  0.  0.4 0.6]
  [0.  0.  0.  1. ]
  [0.  0.  0.  1. ]]]


In [ ]:
# 全てのモデルについて実行する
for save_path, model in model_list:
    # 結果格納用のフォルダ作成
    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    # モデルの要約を出力
    model.summary()

    # モデルのコンパイル
    model.compile(optimizer = opt,        # 最適化手法
            loss = 'mean_squared_error',           # 損失関数
            metrics = ['accuracy']) # 評価関数

    # 途中で保存するモデル基準設定
    fpath = save_path + '/weights.{epoch:03d}-{loss:.2f}-{accuracy:.2f}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5'
    model_ckp = ModelCheckpoint(filepath = fpath, monitor='loss', verbose=1, save_best_only=True, mode='auto', period=5)

    # 学習
    history = model.fit(X_train, Y_train, 
                        epochs=50, 
                        batch_size=10, 
                        validation_split=0.1, 
                        callbacks=[model_ckp])
    
    #Accuracyの推移
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy : ' + save_path)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(save_path + '/accuracy.png')
    plt.close()

    # Lossの推移
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss : ' + save_path)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(save_path + '/loss.png')

    # モデルと学習結果を保存する
    model_json_str = model.to_json()
    open(save_path + '/Keras_rnn.json', 'w').write(model_json_str)
    model.save_weights(save_path + '/Keras_rnn_weights.h5')

plt.close()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42000     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 42,404
Trainable params: 42,404
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


KeyboardInterrupt: ignored

In [ ]:
!cp -r /content/LSTM_1/ /content/drive/MyDrive/ML_LSTM/
!cp -r /content/LSTM_2/ /content/drive/MyDrive/ML_LSTM/